<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Goal" data-toc-modified-id="Goal-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Goal</a></span></li><li><span><a href="#Var" data-toc-modified-id="Var-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Var</a></span></li><li><span><a href="#Init" data-toc-modified-id="Init-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Init</a></span></li><li><span><a href="#Metadata" data-toc-modified-id="Metadata-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Metadata</a></span></li><li><span><a href="#Merging-cluster-reps" data-toc-modified-id="Merging-cluster-reps-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Merging cluster reps</a></span><ul class="toc-item"><li><span><a href="#Georg-animal-gut" data-toc-modified-id="Georg-animal-gut-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Georg animal gut</a></span></li><li><span><a href="#Multi-study" data-toc-modified-id="Multi-study-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Multi-study</a></span></li></ul></li><li><span><a href="#Clustering-reps" data-toc-modified-id="Clustering-reps-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Clustering reps</a></span><ul class="toc-item"><li><span><a href="#LLMGAG" data-toc-modified-id="LLMGAG-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>LLMGAG</a></span></li><li><span><a href="#Run" data-toc-modified-id="Run-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>Run</a></span></li></ul></li><li><span><a href="#Summary" data-toc-modified-id="Summary-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Summary</a></span><ul class="toc-item"><li><span><a href="#Number-of-gene-clusters" data-toc-modified-id="Number-of-gene-clusters-7.1"><span class="toc-item-num">7.1&nbsp;&nbsp;</span>Number of gene clusters</a></span></li></ul></li><li><span><a href="#sessionInfo" data-toc-modified-id="sessionInfo-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>sessionInfo</a></span></li></ul></div>

# Goal

* Merge all per-study 100% seqID cluster reps from all LLMGAG runs
  * creating a non-redundant dataset
  * multi-study + georg animal gut
* Cluster reps at 100% seqID
* Summarize the merged dataset

# Var

In [7]:
work_dir = file.path('/ebio/abt3_projects/Georg_animal_feces/data/metagenome',
                     'multi-study', 'BioProjects', 'merged', 'linclust100')

# georg animal gut metagenomes
GA_base_out_dir = '/ebio/abt3_projects/databases_no-backup/animal_gut_metagenomes/wOutVertebrata/'
GA_out_dirs = c('MG_assembly_act', 'MG_assembly_amp', 'MG_assembly_rep', 'MG_assembly_ave', 'MG_assembly_mam')

# multi-study metagenomes
MS_base_out_dir = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/multi-study/BioProjects/'
MS_out_dirs = c('PRJEB11755', 'PRJEB20308', 'PRJEB29346',
                'PRJNA336354', 'PRJNA381379', 'PRJNA417359',
                'PRJNA532626', 'PRJNA476660', 'PRJNA485217',
                'PRJNA316560-PRJNA316570', 'PRJEB22765',
                'PRJEB23642', 'PRJEB9357')
MS_meta_file = '/ebio/abt3_projects/Georg_animal_feces/data/metagenome/multi-study/metadata/study_metadata_v2.xlsx'


# params
pipeline_dir = '/ebio/abt3_projects/methanogen_host_evo/bin/llmgag/'
conda_env = 'mmseqs'
threads = 16


# Init

In [15]:
library(dplyr)
library(tidyr)
library(ggplot2)
library(data.table)
library(tidytable)
library(doParallel)
library(LeyLabRMisc)

In [9]:
make_dir(work_dir)
setDTthreads(threads)
df.dims()

Created directory: /ebio/abt3_projects/Georg_animal_feces/data/metagenome/multi-study/BioProjects/merged/linclust100 


# Metadata

In [10]:
MS_meta = readxl::read_excel(MS_meta_file) %>%
    dplyr::select(-Publication_DOI) %>%
    rename('No_samples' = Num_samples_used)
df = data.frame('ProjectID' = c('Georg_animal'),
                'Host' = c('Five classes'),
                'No_samples' = c(290))
MS_meta = rbind(MS_meta, df) %>%
    as.data.table
MS_meta

ProjectID,Host,No_samples
<chr>,<chr>,<dbl>
PRJEB11755,Pig,100
PRJEB20308,Dog,100
⋮,⋮,⋮
PRJNA532626,Black rhinoceros,25
Georg_animal,Five classes,290


# Merging cluster reps

In [28]:
# checking for already existing merged seqs
out_file = file.path(work_dir, 'clusters_rep-seqs.faa')
file.exists(out_file)

if(file.exists(out_file)){
    #file.remove(out_file)
}

[1] FALSE

## Georg animal gut

In [29]:
#'' function for merging fasta of sequences
merge_seqs = function(D, base_dir, out_file){
    F = file.path(base_dir, D, 'LLMGAG', 'cluster', 'linclust', 'clusters_rep-seqs.faa')
    if(!file.exists(F)){
        stop(paste0('Cannot find file: ', F))
    }
    cmd = glue::glue('cat {file} >> {out_file}', file=F, out_file=out_file) 
    bash_job(cmd, conda_env='py3')
}

In [30]:
## WARNING: uncomment to re-merge
plyr::llply(as.list(GA_out_dirs), merge_seqs, base_dir=GA_base_out_dir, out_file=out_file) 

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

In [31]:
# number of merged rep sequences
cmd = glue::glue('grep -c ">" {fasta}', fasta=out_file)
n_seqs = as.numeric(system(cmd, intern=TRUE))
cat('Number of rep sequences:', n_seqs , '\n\n')

Number of rep sequences: 29035083 



## Multi-study

In [32]:
## WARNING: uncomment to re-merge
plyr::llply(as.list(MS_out_dirs), merge_seqs, base_dir=MS_base_out_dir, out_file=out_file) 

[[1]]
NULL

[[2]]
NULL

[[3]]
NULL

[[4]]
NULL

[[5]]
NULL

[[6]]
NULL

[[7]]
NULL

[[8]]
NULL

[[9]]
NULL

[[10]]
NULL

[[11]]
NULL

[[12]]
NULL

[[13]]
NULL

In [33]:
# number of merged rep sequences
cmd = glue::glue('grep -c ">" {fasta}', fasta=out_file)
n_seqs = as.numeric(system(cmd, intern=TRUE))
cat('Number of rep sequences:', n_seqs , '\n\n')

Number of rep sequences: 166557140 



# Clustering reps

## LLMGAG

* starting with merged reps at linclust step in the pipeline
  * moved merged rep seqs to `$WORKDIR/assembly/plass/genes.faa`
  * skipped raw & assembly steps of the pipeline

In [34]:
# config file
cat_file(file.path(work_dir, 'config.yaml'))

#-- I/O --#
# table with sample --> read_file information
#samples_file: /ebio/abt3_projects/Georg_animal_feces/data/metagenome/multi-study/BioProjects/merged/linclust90/samples_filler.txt
samples_file: /ebio/abt3_projects/methanogen_host_evo/bin/llmgag/tests/samples/samples_amy_n6.txt

# output location
output_dir: /ebio/abt3_projects/Georg_animal_feces/data/metagenome/multi-study/BioProjects/merged/linclust100/

# cluster reps
## If a fasta file (AA seqs) provided, the pipeline will start at the linclust step
aa_fasta: /ebio/abt3_projects/Georg_animal_feces/data/metagenome/multi-study/BioProjects/merged/linclust100/clusters_rep-seqs.faa

#-- database --#
## eggnog mapper
eggnog_diamond_db: /ebio/abt3_projects/databases_no-backup/Eggnog/v2/eggnog_proteins.dmnd
eggnog_db: /ebio/abt3_projects/databases_no-backup/Eggnog/v2/eggnog.db
## mmseqs taxonomy
mmseqs_tax_db: /ebio/abt3_projects/databases_no-backup/uniclust/uniclust50/uniclust50_2018_08_consensus
## checkM
checkM_data: /ebio/abt3_

## Run

```
(base) @ rick:/ebio/abt3_projects/methanogen_host_evo/bin/llmgag
screen -L -S llmgag-all ./snakemake_sge.sh /ebio/abt3_projects/Georg_animal_feces/data/metagenome/multi-study/BioProjects/merged/linclust100/config.yaml cluster.json /ebio/abt3_projects/Georg_animal_feces/data/metagenome/multi-study/BioProjects/merged/linclust100/SGE_log 50
```

In [35]:
pipelineInfo(pipeline_dir)

LLMGAG

Ley Lab Metagenome Assembly of Genes (LLMGAG)

* Version: 0.1.4
* Authors:
  * Nick Youngblut <nyoungb2@gmail.com>
* Maintainers:
  * Nick Youngblut <nyoungb2@gmail.com>

--- conda envs ---
==> /ebio/abt3_projects/methanogen_host_evo/bin/llmgag//bin/envs/annotate.yaml <==
channels: !!python/tuple
- bioconda
dependencies:
- pigz
- bioconda::fasta-splitter
- bioconda::eggnog-mapper

==> /ebio/abt3_projects/methanogen_host_evo/bin/llmgag//bin/envs/checkm.yaml <==
channels: !!python/tuple
- bioconda
dependencies:
- python=2
- pigz
- bioconda::prodigal
- bioconda::pplacer
- bioconda::checkm-genome

==> /ebio/abt3_projects/methanogen_host_evo/bin/llmgag//bin/envs/das_tool.yaml <==
channels: !!python/tuple
- r
- bioconda
dependencies:
- pigz
- ruby
- r::r-base
- r::r-data.table
- r::r-domc
- r::r-ggplot2
- bioconda::pullseq
- bioconda::prodigal
- bioconda::blast
- bioconda::diamond
==> /ebio/abt3_projects/methanogen_host_evo/bin/llmgag//bin/envs/dask.yaml <==
channels: !!python/tuple


# Summary

## Number of gene clusters

In [36]:
F = file.path(work_dir, 'cluster', 'linclust', 'clusters_rep-seqs.faa')
cmd = glue::glue('grep -c ">" {fasta}', fasta=F)
n_rep_seqs = system(cmd, intern=TRUE)
cat('Number of cluster rep sequences:', n_rep_seqs, '\n')

Number of cluster rep sequences: 150718125 


# sessionInfo

In [37]:
sessionInfo()

R version 3.6.2 (2019-12-12)
Platform: x86_64-conda_cos6-linux-gnu (64-bit)
Running under: Ubuntu 18.04.4 LTS

Matrix products: default
BLAS/LAPACK: /ebio/abt3_projects/Georg_animal_feces/envs/tidyverse/lib/libopenblasp-r0.3.7.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] parallel  stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
[1] doParallel_1.0.15 iterators_1.0.12  foreach_1.4.7     LeyLabRMisc_0.1.3
[5] tidytable_0.3.2   data.table_1.12.8 ggplot2_3.2.1     tidyr_1.0.0      
[9] dplyr_0.8.3      

loaded via a namespace (and not attached):
 [1] Rcpp_1.0.3       plyr_1.8.5       cellranger_